In [ ]:
!apt-get -qq install poppler-utils tesseract-ocr
%pip install -q --user --upgrade pillow
%pip install -q --upgrade unstructured["all-docs"]
%pip install pymupdf
%pip install langchain-openai
%pip install langchain
%pip install qdrant-client
#%pip install langchain-community
%pip install --upgrade nltk
%pip install -U langchain-community

Selecting previously unselected package poppler-utils.
(Reading database ... 126319 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.8_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.8) ...
Setting up poppler-utils (22.02.0-2ubuntu0.8) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 

In [ ]:
import os
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.auto import partition
from unstructured.embed.openai import OpenAIEmbeddingConfig, OpenAIEmbeddingEncoder
import fitz
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
import nltk
nltk.download('punkt')

/tmp/ipython-input-2-68272029.py:4: DeprecationWarning: unstructured.ingest will be removed in a future version. Functionality moved to the unstructured-ingest project.
  from unstructured.embed.openai import OpenAIEmbeddingConfig, OpenAIEmbeddingEncoder
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
!wget 'https://s21.q4cdn.com/399680738/files/doc_news/Meta-Reports-Second-Quarter-2024-Results-2024.pdf' -O './doc1.pdf'

--2025-06-23 10:30:55--  https://s21.q4cdn.com/399680738/files/doc_news/Meta-Reports-Second-Quarter-2024-Results-2024.pdf
Resolving s21.q4cdn.com (s21.q4cdn.com)... 68.70.205.2, 68.70.205.3, 68.70.205.1, ...
Connecting to s21.q4cdn.com (s21.q4cdn.com)|68.70.205.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 195613 (191K) [application/pdf]
Saving to: ‘./doc1.pdf’

./doc1.pdf          100%[===================>] 191.03K  --.-KB/s    in 0.05s   

2025-06-23 10:30:55 (3.70 MB/s) - ‘./doc1.pdf’ saved [195613/195613]



# Document reading

In [ ]:
elements = partition_pdf('./doc1.pdf',
                              strategy="hi_res",
                              infer_table_structure=True,
                              chunking_strategy="by_title",
                              max_characters=400,
                              combine_text_under_n_chars=50,
                              new_after_n_chars=200
                              )

In [ ]:
from collections import Counter
display(Counter(type(element) for element in elements))

Counter({unstructured.documents.elements.CompositeElement: 57,
         unstructured.documents.elements.TableChunk: 12})

In [ ]:
from IPython.display import Markdown, display, HTML

for element in elements:
  if element.to_dict()['type']=="Table":
    if "is_continuation" in element.to_dict()["metadata"]:
      html_table = element.metadata.text_as_html
      display(HTML(html_table))
    else:
      print("----------------------NEW_TABLE----------------------")
      page_no = element.to_dict()["metadata"]["page_number"]
      print("Page no:", page_no)
      html_table = element.metadata.text_as_html
      display(HTML(html_table))
  else:
    display(Markdown(str(element.text)))

FACEBOOK

NEWS RELEASE

Meta Reports Second Quarter 2024 Results

MENLO PARK, Calif., July 31, 2024 /PRNewswire/ -- Meta Platforms, Inc. (Nasdaq: META) today reported nancial results for the quarter ended June 30, 2024.

"We had a strong quarter, and Meta AI is on track to be the most used AI assistant in the world by the end of the year," said Mark Zuckerberg, Meta founder and CEO. "We've released the rst frontier-level open source AI model, we continue to see good traction with our Ray-Ban Meta AI glasses, and we're driving good growth across our apps."

Second Quarter 2024 Financial Highlights

Three Months Ended June 30, In millions, except percentages and per share amounts 2024 2023 % Change Revenue $ 39,071 $ 31,999 22 % Costs and expenses 24,224 22,607 7 % Income from operations $ 14,847 $ 9,392 58 % Operating margin 38 % 29 % Provision for income taxes $ 1,641 $ 1,505 9 % Effective tax rate 11 % 16 % Net income $ 13,465 $ 7,788 73 % Diluted earnings per share (EPS) $ 5.16 $ 2.98 73 %

Second Quarter 2024 Operational and Other Financial Highlights

Family daily active people (DAP) – DAP was 3.27 billion on average for June 2024, an increase of 7% year-over- year.

Ad impressions – Ad impressions delivered across our Family of Apps increased by 10% year-over-year.

e

e

Average price per ad – Average price per ad increased by 10% year-over-year.

Revenue – Total revenue was $39.07 billion, an increase of 22% year-over-year. Revenue on a constant

...

(Unaudited)

Three Months Ended June 30, Six Months Ended June 30,

----------------------NEW_TABLE----------------------
Page no: 9


GAAP revenue

$ 60,645

$ 59,599

change %

$ 31,307

(12,957)

(484)

Free cash flow

$ 17,866

View original content to download multimedia:https://www.prnewswire.com/news-releases/meta-reports- second-quarter-2024-results-302211512.html

SOURCE Meta

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

pdf_path = './doc1.pdf'
document_content = extract_text_from_pdf(pdf_path)

In [ ]:
display(Markdown(str(document_content)))

NEWS RELEASE
Meta Reports Second Quarter 2024 Results
MENLO PARK, Calif., July 31, 2024 /PRNewswire/ -- Meta Platforms, Inc. (Nasdaq: META) today reported ínancial
results for the quarter ended June 30, 2024.
"We had a strong quarter, and Meta AI is on track to be the most used AI assistant in the world by the end of the
year," said Mark Zuckerberg, Meta founder and CEO. "We've released the írst frontier-level open source AI model,
we continue to see good traction with our Ray-Ban Meta AI glasses, and we're driving good growth across our
apps."
 
 
 Second Quarter 2024 Financial Highlights
 
Three Months Ended June 30,
 
 % Change
In millions, except percentages and per share amounts               
2024
 
2023
 
Revenue
$                  39,071  
$                  31,999  
22 %
Costs and expenses
24,224  
22,607  
7 %
Income from operations
$                  14,847  
$                    9,392  
58 %
Operating margin
38 %  
29 %  
 
Provision for income taxes
$                    1,641  
$                    1,505  
9 %
Effective tax rate
11 %  
16 %  
 
Net income
$                  13,465  
$                    7,788  
73 %
Diluted earnings per share (EPS)
$                      5.16  
$                      2.98  
73 %
 
 Second Quarter 2024 Operational and Other Financial Highlights
Family daily active people (DAP) – DAP was 3.27 billion on average for June 2024, an increase of 7% year-over-
year.
Ad impressions – Ad impressions delivered across our Family of Apps increased by 10% year-over-year.
Average price per ad – Average price per ad increased by 10% year-over-year.
Revenue – Total revenue was $39.07 billion, an increase of 22% year-over-year. Revenue on a constant
1
currency basis would have increased 23% year-over-year.
Costs and expenses – Total costs and expenses were $24.22 billion, an increase of 7% year-over-year.
Capital expenditures – Capital expenditures, including principal payments on ínance leases, were $8.47
billion.
Capital return program – Share repurchases were $6.32 billion of our Class A common stock and dividend
payments were $1.27 billion.
Cash, cash equivalents, and marketable securities – Cash, cash equivalents, and marketable securities were
$58.08 billion as of June 30, 2024. Free cash îow was $10.90 billion.
Headcount – Headcount was 70,799 as of June 30, 2024, a decrease of 1% year-over-year.
 
CFO Outlook Commentary
 
 
We expect third quarter 2024 total revenue to be in the range of $38.5-41 billion. Our guidance assumes foreign
currency is a 2% headwind to year-over-year total revenue growth, based on current exchange rates.
We expect full-year 2024 total expenses to be in the range of $96-99 billion, unchanged from our prior outlook. For
Reality Labs, we continue to expect 2024 operating losses to increase meaningfully year-over-year due to our
ongoing product development eìorts and investments to further scale our ecosystem.
While we do not intend to provide any quantitative guidance for 2025 until the fourth quarter call, we expect
infrastructure costs will be a signiícant driver of expense growth next year as we recognize depreciation and
operating costs associated with our expanded infrastructure footprint.
We anticipate our full-year 2024 capital expenditures will be in the range of $37-40 billion, updated from our prior
range of $35-40 billion. While we continue to reíne our plans for next year, we currently expect signiícant capital
expenditures growth in 2025 as we invest to support our artiícial intelligence research and product development
eìorts.
Absent any changes to our tax landscape, we expect our full-year 2024 tax rate to be in the mid-teens.
In addition, we continue to monitor an active regulatory landscape, including the increasing legal and regulatory
headwinds in the EU and the U.S. that could signiícantly impact our business and our ínancial results.
 
Webcast and Conference Call Information
 
 
Meta will host a conference call to discuss the results at 2:00 p.m. PT / 5:00 p.m. ET today. The live webcast of
2
Meta's earnings conference call can be accessed at the Meta Investor Relations website at investor.fb.com, along
with the earnings press release, ínancial tables, and slide presentation. 
Following the call, a replay will be available at the same website. Transcripts of conference calls with publishing
equity research analysts held today will also be posted to the investor.fb.com website.
 
 
Disclosure Information
 
 
Meta uses the investor.fb.com and about.fb.com/news/ websites as well as Mark Zuckerberg's Facebook Page
(facebook.com/zuck), Instagram account (instagram.com/zuck) and Threads proíle (threads.net/zuck) as means
of disclosing material non-public information and for complying with its disclosure obligations under Regulation FD.
 
 
About Meta
 
 
Meta builds technologies that help people connect, índ communities, and grow businesses. When Facebook
launched in 2004, it changed the way people connect. Apps like Messenger, Instagram, and WhatsApp further
empowered billions around the world. Now, Meta is moving beyond 2D screens toward immersive experiences like
augmented and virtual reality to help build the next evolution in social technology.
 
 
Contacts
 
 
Investors:
 
Kenneth Dorell
 
investor@meta.com / investor.fb.com
Press:
 
Ryan Moore
 
press@meta.com / about.fb.com/news/
 
 
Forward-Looking Statements
 
 
3
This press release contains forward-looking statements regarding our future business plans and expectations.
These forward-looking statements are only predictions and may diìer materially from actual results due to a variety
of factors including: the impact of macroeconomic conditions on our business and ínancial results, including as a
result of geopolitical events; our ability to retain or increase users and engagement levels; our reliance on
advertising revenue; our dependency on data signals and mobile operating systems, networks, and standards that
we do not control; changes to the content or application of third-party policies that impact our advertising
practices; risks associated with new products and changes to existing products as well as other new business
initiatives, including our artiícial intelligence initiatives and metaverse eìorts; our emphasis on community growth
and engagement and the user experience over short-term ínancial results; maintaining and enhancing our brand
and reputation; our ongoing privacy, safety, security, and content review eìorts; competition; risks associated with
government actions that could restrict access to our products or impair our ability to sell advertising in certain
countries; litigation and government inquiries; privacy, legislative, and regulatory concerns or developments; risks
associated with acquisitions; security breaches; our ability to manage our scale and geographically-dispersed
operations; and market conditions or other factors aìecting the payment of dividends. These and other potential
risks and uncertainties that could cause actual results to diìer from the results predicted are more fully detailed
under the caption "Risk Factors" in our Quarterly Report on Form 10-Q íled with the SEC on April 25, 2024, which is
available on our Investor Relations website at investor.fb.com and on the SEC website at www.sec.gov. Additional
information will also be set forth in our Quarterly Report on Form 10-Q for the quarter ended June 30, 2024. In
addition, please note that the date of this press release is July 31, 2024, and any forward-looking statements
contained herein are based on assumptions that we believe to be reasonable as of this date. We undertake no
obligation to update these statements as a result of new information or future events.
For a discussion of limitations in the measurement of certain of our community metrics, see the section entitled
"Limitations of Key Metrics and Other Data" in our most recent quarterly or annual report íled with the SEC.
Non-GAAP Financial Measures
To supplement our condensed consolidated ínancial statements, which are prepared and presented in accordance
with generally accepted accounting principles in the United States (GAAP), we use the following non-GAAP ínancial
measures: revenue excluding foreign exchange eìect, advertising revenue excluding foreign exchange eìect, and
free cash îow. The presentation of these ínancial measures is not intended to be considered in isolation or as a
substitute for, or superior to, ínancial information prepared and presented in accordance with GAAP. Investors are
cautioned that there are material limitations associated with the use of non-GAAP ínancial measures as an
analytical tool. In addition, these measures may be diìerent from non-GAAP ínancial measures used by other
companies, limiting their usefulness for comparison purposes. We compensate for these limitations by providing
speciíc information regarding the GAAP amounts excluded from these non-GAAP ínancial measures.
We believe these non-GAAP ínancial measures provide investors with useful supplemental information about the
4
ínancial performance of our business, enable comparison of ínancial results between periods where certain items
may vary independent of business performance, and allow for greater transparency with respect to key metrics
used by management in operating our business.
Our non-GAAP ínancial measures are adjusted for the following items:
Foreign exchange eìect on revenue. We translated revenue for the three and six months ended June 30, 2024 using
the prior year's monthly exchange rates for our settlement or billing currencies other than the U.S. dollar, which we
believe is a useful metric that facilitates comparison to our historical performance.
Purchases of property and equipment; Principal payments on ínance leases. We subtract both purchases of
property and equipment, net of proceeds and principal payments on ínance leases in our calculation of free cash
îow because we believe that these two items collectively represent the amount of property and equipment we
need to procure to support our business, regardless of whether we procure such property or equipment with a
ínance lease. We believe that this methodology can provide useful supplemental information to help investors
better understand underlying trends in our business. Free cash îow is not intended to represent our residual cash
îow available for discretionary expenditures.
For more information on our non-GAAP ínancial measures and a reconciliation of GAAP to non-GAAP measures,
please see the "Reconciliation of GAAP to Non-GAAP Results" table in this press release.
 
META PLATFORMS, INC.
CONDENSED CONSOLIDATED STATEMENTS OF INCOME
(In millions, except per share amounts)
(Unaudited)
 
Three Months Ended June 30,  Six Months Ended June 30,
 
2024
 
2023
 
2024
 
2023
Revenue
$           39,071  
$           31,999  $           75,527  $           60,645
Costs and expenses:
  
  
  
 
Cost of revenue
7,308  
5,945  
13,948  
12,054
Research and development
10,537  
9,344  
20,515  
18,725
Marketing and sales
2,721  
3,154  
5,285  
6,198
General and administrative (1)
3,658  
4,164  
7,114  
7,049
Total costs and expenses
24,224  
22,607  
46,862  
44,026
Income from operations
14,847  
9,392  
28,665  
16,619
Interest and other income (expense), net
259  
(99)  
624  
(19)
Income before provision for income taxes
15,106  
9,293  
29,289  
16,600
Provision for income taxes
1,641  
1,505  
3,455  
3,102
Net income
$           13,465  
$              7,788  $           25,834  $           13,498
5
Earnings per share:
  
  
  
 
Basic
$               5.31  
$                3.03  $             10.17  $               5.24
Diluted
$               5.16  
$                2.98  $               9.86  $               5.18
Weighted-average shares used to compute earnings per share:   
  
  
  
 
Basic
2,534  
2,568  
2,540  
2,577
Diluted
2,610  
2,612  
2,619  
2,604
 
 
 
 
 
(1) The second quarter 2024 general and administrative expenses include a charge for the recent settlement with the State of Texas. The
settlement amount is fully accrued as of June 30, 2024.
 
META PLATFORMS, INC.
CONDENSED CONSOLIDATED BALANCE SHEETS
(In millions)
(Unaudited)
 
June 30, 2024
 December 31, 2023
Assets
  
 
Current assets:
  
 
Cash and cash equivalents
$                  32,045  
$                  41,862
Marketable securities
26,035  
23,541
Accounts receivable, net
14,505  
16,169
Prepaid expenses and other current assets                            
3,846  
3,793
Total current assets
76,431  
85,365
Non-marketable equity securities
6,207  
6,141
Property and equipment, net
102,959  
96,587
Operating lease right-of-use assets
14,058  
13,294
Goodwill
20,654  
20,654
Other assets
9,929  
7,582
Total assets
$                230,238  
$                229,623
 
  
 
Liabilities and stockholders' equity
  
 
Current liabilities:
  
 
Accounts payable
$                    3,173  
$                    4,849
Operating lease liabilities, current
1,917  
1,623
Accrued expenses and other current liabilities                                                 
21,914  
25,488
Total current liabilities
27,004  
31,960
Operating lease liabilities, non-current
17,685  
17,226
Long-term debt
18,389  
18,385
Long-term income taxes
7,897  
7,514
Other liabilities
2,500  
1,370
Total liabilities
73,475  
76,455
Commitments and contingencies
  
 
Stockholders' equity:
  
 
Common stock and additional paid-in capital
78,270  
73,253
6
Accumulated other comprehensive loss
(2,695)  
(2,155)
Retained earnings
81,188  
82,070
Total stockholders' equity
156,763  
153,168
Total liabilities and stockholders' equity
$                230,238  
$                229,623
 
META PLATFORMS, INC.
CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS
(In millions)
(Unaudited)
 
Three Months Ended June 30,  Six Months Ended June 30,
 
2024
 
2023
 
2024
 
2023
Cash flows from operating activities
  
  
  
 
Net income
$      13,465  
$        7,788  
$      25,834  
$      13,498
Adjustments to reconcile net income to net cash provided by operating activities:
  
  
  
 
Depreciation and amortization
3,637  
2,623  
7,011  
5,147
Share-based compensation
4,616  
4,060  
8,178  
7,111
Deferred income taxes
(1,643)  
(1,137)  
(2,098)  
(1,757)
Impairment charges for facilities consolidation, net
41  
232  
280  
1,002
Other
(6)  
212  
(71)  
204
Changes in assets and liabilities:
  
  
  
 
Accounts receivable
(1,171)  
(1,424)  
1,350  
1,122
Prepaid expenses and other current assets
(84)  
(54)  
16  
767
Other assets
54  
37  
(41)  
67
Accounts payable
250  
(51)  
(862)  
(1,155)
Accrued expenses and other current liabilities
(497)  
5,174  
(1,771)  
5,268
Other liabilities
708  
(151)  
790  
33
Net cash provided by operating activities
19,370  
17,309  
38,616  
31,307
Cash flows from investing activities
  
  
  
 
Purchases of property and equipment, net
(8,173)  
(6,134)  
(14,573)  
(12,957)
Purchases of marketable debt securities
(3,289)  
(717)  
(10,176)  
(803)
Sales and maturities of marketable debt securities
3,233  
1,816  
7,858  
2,351
Acquisitions of businesses and intangible assets
(57)  
(83)  
(129)  
(527)
Other investing activities
(12)  
(85)  
(12)  
(10)
Net cash used in investing activities
(8,298)  
(5,203)  
(17,032)  
(11,946)
Cash flows from financing activities
  
  
  
 
Taxes paid related to net share settlement of equity awards
(3,208)  
(1,692)  
(6,370)  
(2,701)
Repurchases of Class A common stock
(6,299)  
(898)  
(21,307)  
(10,263)
Dividend payments
(1,266)  
—  
(2,539)  
—
Proceeds from issuance of long-term debt, net
—  
8,455  
—  
8,455
Principal payments on finance leases
(299)  
(220)  
(614)  
(484)
Other financing activities
(106)  
(353)  
(115)  
(231)
Net cash provided by (used in) financing activities
(11,178)  
5,292  
(30,945)  
(5,224)
Effect of exchange rate changes on cash, cash equivalents, and restricted cash
(152)  
(14)  
(440)  
71
Net increase (decrease) in cash, cash equivalents, and restricted cash
(258)  
17,384  
(9,801)  
14,208
Cash, cash equivalents, and restricted cash at beginning of the period
33,284  
12,420  
42,827  
15,596
Cash, cash equivalents, and restricted cash at end of the period
$      33,026  
$      29,804  
$      33,026  
$      29,804
7
 
  
  
  
 
Reconciliation of cash, cash equivalents, and restricted cash to the
 
condensed consolidated balance sheets
  
  
  
 
Cash and cash equivalents
$      32,045  
$      28,785  
$      32,045  
$      28,785
Restricted cash, included in prepaid expenses and other current assets
100  
165  
100  
165
Restricted cash, included in other assets
881  
854  
881  
854
Total cash, cash equivalents, and restricted cash
$      33,026  
$      29,804  
$      33,026  
$      29,804
 
META PLATFORMS, INC.
CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS
(In millions)
(Unaudited)
 
Three Months Ended June 30,  Six Months Ended June 30,
 
2024
 
2023
 
2024
 
2023
Supplemental cash flow data
  
  
  
 
Cash paid for income taxes, net
$        5,929
 
$        1,102  
$        6,559  
$        1,507
Cash paid for interest, net of amounts capitalized
$           124  
$             —  
$           245  
$           182
Non-cash investing and financing activities:
  
  
  
 
Property and equipment in accounts payable and accrued expenses and other
 
current liabilities
$        3,229
 
$        3,845  
$        3,229  
$        3,845
Acquisition of businesses and intangible assets in accrued expenses and
 
other current liabilities and other liabilities
$           267  
$           217  
$           267  
$           217
 
Segment Results
We report our ínancial results for our two reportable segments: Family of Apps (FoA) and Reality Labs (RL). FoA
includes Facebook, Instagram, Messenger, WhatsApp, and other services. RL includes our virtual, augmented, and
mixed reality related consumer hardware, software, and content.
The following table presents our segment information of revenue and income (loss) from operations:
Segment Information
(In millions)
(Unaudited)
 
Three Months Ended June 30,   Six Months Ended June 30,
 
2024
  
2023
  
2024
  
2023
Revenue:
   
   
   
 
8
Advertising
$      38,329   
$      31,498   
$     73,965   
$     59,599
Other revenue
389   
225   
769   
430
Family of Apps
38,718   
31,723   
74,734   
60,029
Reality Labs
353   
276   
793   
616
Total revenue
$      39,071   
$      31,999   
$     75,527   
$     60,645
 
   
   
   
 
Income (loss) from operations:
   
   
   
 
Family of Apps
$      19,335   
$      13,131   
$     36,999   
$     24,351
Reality Labs
(4,488)   
(3,739)   
(8,334)   
(7,732)
Total income from operations                                           
$      14,847   
$        9,392   
$     28,665   
$     16,619
 
Reconciliation of GAAP to Non-GAAP Results
(In millions, except percentages)
(Unaudited)
 
Three Months Ended June 30,  Six Months Ended June 30,
 
2024
 
2023
 
2024
 
2023
GAAP revenue
$      39,071  
$      31,999  
$      75,527  
$       60,645
Foreign exchange effect on 2024 revenue using 2023 rates
371  
  
265  
 
Revenue excluding foreign exchange effect
$      39,442  
  
$      75,792  
 
GAAP revenue year-over-year change %
22 %
 
  
25 %  
 
Revenue excluding foreign exchange effect year-over-year change %
23 %
 
  
25 %  
 
GAAP advertising revenue
$      38,329  
$      31,498  
$      73,965  
$       59,599
Foreign exchange effect on 2024 advertising revenue using 2023 rates             
367  
  
261  
 
Advertising revenue excluding foreign exchange effect
$      38,696  
  
$      74,226  
 
GAAP advertising revenue year-over-year change %
22 %
 
  
24 %  
 
Advertising revenue excluding foreign exchange effect year-over-year
 
change %
23 %
 
  
25 %  
 
 
  
  
  
 
Net cash provided by operating activities
$      19,370  
$      17,309  
$      38,616  
$       31,307
Purchases of property and equipment, net
(8,173)  
(6,134)  
(14,573)  
(12,957)
Principal payments on finance leases
(299)  
(220)  
(614)  
(484)
Free cash flow
$      10,898  
$      10,955  
$      23,429  
$       17,866
 
View original content to download multimedia:https://www.prnewswire.com/news-releases/meta-reports-
second-quarter-2024-results-302211512.html
SOURCE Meta
9


In [ ]:
len(document_content)

20345

In [ ]:
document_content[:1000]

'NEWS RELEASE\nMeta Reports Second Quarter 2024 Results\nMENLO PARK, Calif., July 31, 2024 /PRNewswire/ -- Meta Platforms, Inc. (Nasdaq: META) today reported ínancial\nresults for the quarter ended June\xa030, 2024.\n"We had a strong quarter, and Meta AI is on track to be the most used AI assistant in the world by the end of the\nyear," said Mark Zuckerberg, Meta founder and CEO. "We\'ve released the írst frontier-level open source AI model,\nwe continue to see\xa0good traction with our Ray-Ban Meta AI glasses, and we\'re driving good growth across our\napps."\n \n \n\xa0Second Quarter 2024\xa0Financial Highlights\n \nThree Months Ended June 30,\n \n % Change\nIn millions, except percentages and per share amounts               \n2024\n \n2023\n \nRevenue\n$                  39,071  \n$                  31,999  \n22 %\nCosts and expenses\n24,224  \n22,607  \n7 %\nIncome from operations\n$                  14,847  \n$                    9,392  \n58 %\nOperating margin\n38 %  \n29 %  \n \

In [ ]:
# 8191 - embedding model
# 1,28,000 - gpt-4o-mini

# Document Table Processing

In [ ]:
import openai
from openai import OpenAI
client = OpenAI()

def table_chunking(table_content, document_context):
    prompt = f"""
      You are a table restoration and summarization assistant.

      You will be given:
      - An HTML table chunk and
      - Its context from the original document.

      Your task:
      1. Restore missing column headers using the document context.
      2. Correct any corrupted values from the context.
      3. Return the table in markdown format.
      4. Describe the table briefly and assign it a semantic title.
      5. If the column headers are split across multiple rows, flatten them into single-line column names
        For example, if the table has headers like:

      |                     | Three Months Ended June 30 | Six Months Ended June 30 |
      |---------------------|----------------------------|--------------------------|
      |                     | 2024 |   2023              | 2024 | 2023              |

      Then flatten it into:

      | Three Months Ended June 30 2024 | Three Months Ended June 30 2023 | Six Months Ended June 30 2024 | Six Months Ended June 30 2023 |


      Original Document Context:
      {document_context}

      HTML Table Chunk:
      {table_content}

      Return your answer strictly in the following format:

      TableDescription: <Brief natural language description of what the table contains>
      TableHeader: <Semantic title for the table, e.g., "Quarterly Financial Summary">
      TableMarkdown: <Markdown formatted table>

      Answer:
      """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that describes tables and formats them in markdown."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content

In [ ]:
from IPython.display import Markdown, display, HTML

c=0
for element in elements:
  if element.to_dict()['type']=="Table":
    # c+=1
    # if c>5:
    #   break
    if "is_continuation" in element.to_dict()["metadata"]:
      html_table = element.metadata.text_as_html
      result = table_chunking(html_table,document_content)
      print(result)
      element.text = result
    else:
      print("----------------------NEW_TABLE----------------------")
      page_no = element.to_dict()["metadata"]["page_number"]
      print("Page no:", page_no)
      html_table = element.metadata.text_as_html
      result = table_chunking(html_table,document_content)
      element.text = result
      print(result)

----------------------NEW_TABLE----------------------
Page no: 5
TableDescription: This table summarizes the condensed consolidated statements of income for Meta Platforms, Inc. for the three and six months ended June 30, 2024 and 2023, including revenues, costs and expenses, income from operations, and related financial figures in millions of dollars.

TableHeader: Quarterly Financial Summary

TableMarkdown:
```markdown
| Revenue                          | 2024       | 2023       | 2024       | 2023       |
|----------------------------------|------------|------------|------------|------------|
| Revenue                          | $39,071   | $31,999   | $75,527   | $60,645   |
| Costs and expenses:              |            |            |            |            |
| Cost of revenue                  | $7,308    | $5,945    | $13,948   | $12,054   |
| Research and development          | $10,537   | $9,344    | $20,515   | $18,725   |
| Marketing and sales              | $2,721    | $3,

# Embedding

In [ ]:
from unstructured.embed.openai import OpenAIEmbeddingConfig, OpenAIEmbeddingEncoder

embedding_encoder = OpenAIEmbeddingEncoder(
    config=OpenAIEmbeddingConfig(
      model_name="text-embedding-3-small",
      api_key = OPENAI_API_KEY
    )
)
elements = embedding_encoder.embed_documents(
    elements=elements
)

# Qdrant

In [ ]:
from qdrant_client.models import PointStruct
import uuid

points = []

for idx, element in enumerate(elements):
    vector = element.embeddings
    if vector is None:
        continue  # Skip unembedded

    point = PointStruct(
        id=str(uuid.uuid4()),
        vector=vector,
        payload={
            "text": element.text,
            "metadata": element.metadata.to_dict(),
        }
    )
    points.append(point)

In [ ]:
from qdrant_client import QdrantClient

qdrant_cloud_key = userdata.get('QDRANT_CLOUD_KEY')

client = QdrantClient(
    url="https://01f14d7a-9e05-4804-bfd3-a69b4c3a97da.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=qdrant_cloud_key,
)

print(client.get_collections())

collections=[CollectionDescription(name='meta_report')]


In [ ]:
from qdrant_client.models import VectorParams, Distance

collection_name = "meta_report_tables"

if collection_name not in [col.name for col in client.get_collections().collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=len(points[0].vector), distance=Distance.COSINE)
    )

/tmp/ipython-input-24-2263194056.py:6: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


In [ ]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# RAG Pipeline

## Retreiver

In [34]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings  # or any embedding model you're using

# Optional: if you already have embeddings, this just helps for retriever
embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")

# Wrap Qdrant client with LangChain's Qdrant vector store
vecdb = Qdrant(
    client=client,
    collection_name=collection_name,
    embeddings=embedding_function,
    content_payload_key="text"
)

/tmp/ipython-input-34-339356876.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_function = OpenAIEmbeddings(model="text-embedding-3-small")
/tmp/ipython-input-34-339356876.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vecdb = Qdrant(


In [ ]:
# At the end of 2023, what was the value of Meta's Goodwill assets?
results = vecdb.similarity_search("what is the 2024 GAAP advertising Revenue in the three months \
     ended June 30th? What about net cash by operating activies", k=5)
for doc in results:
    print(doc.page_content)

TableDescription: This table presents key financial metrics for Meta Platforms, Inc., comparing GAAP and non-GAAP advertising revenue, operating activities, purchases of property and equipment, and finance lease payments for two time periods: the most recent quarters and the corresponding prior year quarters.

TableHeader: Advertising Revenue and Cash Flow Metrics

TableMarkdown: 
```markdown
| Metric                                                              | Q2 2024        | Q2 2023     | Six Months 2024 | Six Months 2023 |
|---------------------------------------------------------------------|----------------|-------------|------------------|------------------|
| Advertising revenue excluding foreign exchange effect                 | $38,696        | -           | $74,226          | -                |
| GAAP advertising revenue year-over-year change %                     | 22%            | -           | 24%              | -                |
| Advertising revenue excluding foreign

## RAG

In [36]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        You are an intelligent assistant. Use the following context to answer the question at the end.

        Context:
        {context}

        Question:
        {question}

        Answer:"""
        )

llm = ChatOpenAI(model_name="gpt-4o-mini")
qa_chain = LLMChain(llm=llm, prompt=prompt_template)

def rag(query: str, k: int = 5):
    retrieved_docs = vecdb.similarity_search(query, k=k)

    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    result = qa_chain.run(context=context, question=query)

    return {
        "answer": result,
        "source_documents": retrieved_docs
    }

In [37]:
# Example usage
response = rag("what is the 2024 GAAP advertising Revenue in the three months ended June 30th? What about net cash by operating activies")
print("Answer:", response["answer"])

/tmp/ipython-input-35-1474145917.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(context=context, question=query)


Answer: The GAAP advertising revenue for the three months ended June 30, 2024, is $38,329 million. The net cash provided by operating activities for the same period is $19,370 million.


# Evaluation

In [29]:
%pip install ares-ai
%pip install ragas
%pip install qdrant-client
%pip install langchain-openai
%pip install langchain
%pip install -U langchain-community
%pip install datasets

In [18]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## Create synthetic data

In [14]:
import uuid
from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue, SearchParams
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from google.colab import userdata

qdrant_cloud_key = userdata.get('QDRANT_CLOUD_KEY')
client = QdrantClient(
    url="https://01f14d7a-9e05-4804-bfd3-a69b4c3a97da.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=qdrant_cloud_key
)

collection_name = "meta_report_tables"

points = client.scroll(
    collection_name=collection_name,
    scroll_filter=None,
    limit=100,
)

table_points = [
    record for record in points[0]
    if 'metadata' in record.payload and 'text_as_html' in record.payload['metadata']
]

In [22]:
table_points[0].payload["text"]

'Second Quarter 2024 Financial Highlights\n\nThree Months Ended June 30, In millions, except percentages and per share amounts 2024 2023 % Change Revenue $ 39,071 $ 31,999 22 % Costs and expenses 24,224 22,607 7 % Income from operations $ 14,847 $ 9,392 58 % Operating margin 38 % 29 % Provision for income taxes $ 1,641 $ 1,505 9 % Effective tax rate 11 % 16 % Net income $ 13,465 $ 7,788 73 % Diluted earnings per share (EPS) $ 5.16 $ 2.98 73 %'

In [23]:
from langchain_core.runnables import RunnableLambda
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from typing import List
from pydantic import BaseModel, Field

class QAPair(BaseModel):
    question: str = Field(..., description="A question based on the context")
    answer: str = Field(..., description="The answer to the question based on the context")

class QAPairList(BaseModel):
    context: str = Field(..., description="The document context")
    context_id: str = Field(..., description="Unique identifier of the context")
    qa_pairs: List[QAPair] = Field(..., description="A list of QA pairs generated from the context")

parser = PydanticOutputParser(pydantic_object=QAPairList)

prompt = PromptTemplate.from_template(
    """You are a QA generator.

    Context:
    {context}

    Generate 3 diverse and relevant question-answer pairs from the above context.
    Format:
    {format_instructions}
    """
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

qa_chain = (
    prompt |
    llm |
    parser
)

synthetic_data = []

for point in table_points:
    context_text = point.payload["text"]
    context_id = str(point.id)

    result = qa_chain.invoke({
        "context": context_text,
        "format_instructions": parser.get_format_instructions()
    })

    result.context = context_text
    result.context_id = context_id
    synthetic_data.append(result)
    break

In [27]:
print(synthetic_data[0].qa_pairs)

[QAPair(question='What was the revenue for the second quarter of 2024?', answer='$39,071 million'), QAPair(question='How much did the net income increase from 2023 to 2024 in percentage terms?', answer='73%'), QAPair(question='What was the operating margin for the second quarter of 2024?', answer='38%')]


In [28]:
all_qa = []
for item in synthetic_data:
    for qa in item.qa_pairs:
        all_qa.append({
            "question": qa.question,
            "answer": qa.answer,
            "context": item.context,
            "context_id": item.context_id
        })

In [30]:
from datasets import Dataset

hf_dataset = Dataset.from_list(all_qa)

In [32]:
hf_dataset.push_to_hub("JC-24/meta-record-rag-synthetic-dataset", token=userdata.get("HF_TOKEN"))

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [33]:
hf_dataset

Dataset({
    features: ['question', 'answer', 'context', 'context_id'],
    num_rows: 3
})

In [51]:
# from datasets import load_dataset

# dataset = load_dataset("JC-24/meta-record-rag-synthetic-dataset")

In [52]:
hf_dataset[0]

{'question': 'What was the revenue for the second quarter of 2024?',
 'answer': '$39,071 million',
 'context': 'Second Quarter 2024 Financial Highlights\n\nThree Months Ended June 30, In millions, except percentages and per share amounts 2024 2023 % Change Revenue $ 39,071 $ 31,999 22 % Costs and expenses 24,224 22,607 7 % Income from operations $ 14,847 $ 9,392 58 % Operating margin 38 % 29 % Provision for income taxes $ 1,641 $ 1,505 9 % Effective tax rate 11 % 16 % Net income $ 13,465 $ 7,788 73 % Diluted earnings per share (EPS) $ 5.16 $ 2.98 73 %',
 'context_id': '00ce17c7-1a61-47d5-937f-78a001552ba4'}

## Ragas

In [64]:
from datasets import Dataset, load_dataset

# dataset_path="JC-24/meta-record-rag-synthetic-dataset"
# hf_dataset = load_dataset(dataset_path, token=userdata.get("HF_TOKEN"))

ragas_dataset = []
for item in hf_dataset:
  query = item["question"]
  response = rag(query)
  source_documents = response["source_documents"]
  answer = response["answer"]
  ragas_dataset.append(
      {
          "question": item["question"],
          "ground_truth": item["answer"],
          "context": [item["context"]],
          "answer":answer,
          "retrieved_contexts":[d.page_content for d in source_documents]
      }
  )

ragas_df = pd.DataFrame(ragas_dataset)
ragas_df.to_csv("rag_evaluated_dataset.csv", index=False)
ragas_dataset_hf = Dataset.from_list(ragas_dataset)
# for data in ragas_dataset:
#   data["retrieved_contexts"] = [d.page_content for d in data["retrieved_contexts"]]

In [71]:
from ragas.evaluation import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)

result = evaluate(
    ragas_dataset_hf,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall
    ]
)
print(result)

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

{'faithfulness': 0.7333, 'answer_relevancy': 0.9863, 'context_precision': 0.8000, 'context_recall': 1.0000}


In [77]:
for data in ragas_dataset:
  print("\n\nDATA-----------------------------------------#################################--------------------------------------")
  for col in data.keys():
    if col=="retrieved_contexts":
      print("*",col,":")
      for l in data[col]:
        print("#######################################################")
        print(l)
    else:
      print("*",col, ": ",data[col])



DATA-----------------------------------------#################################--------------------------------------
* question :  What was the revenue for the second quarter of 2024?
* ground_truth :  $39,071 million
* context :  ['Second Quarter 2024 Financial Highlights\n\nThree Months Ended June 30, In millions, except percentages and per share amounts 2024 2023 % Change Revenue $ 39,071 $ 31,999 22 % Costs and expenses 24,224 22,607 7 % Income from operations $ 14,847 $ 9,392 58 % Operating margin 38 % 29 % Provision for income taxes $ 1,641 $ 1,505 9 % Effective tax rate 11 % 16 % Net income $ 13,465 $ 7,788 73 % Diluted earnings per share (EPS) $ 5.16 $ 2.98 73 %']
* answer :  The revenue for the second quarter of 2024 was $39,071 million.
* retrieved_contexts :
#######################################################
Second Quarter 2024 Financial Highlights

Three Months Ended June 30, In millions, except percentages and per share amounts 2024 2023 % Change Revenue $ 39,071 $ 

# MLFlow

In [78]:
%pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.0 MB/s eta 0:00:00


In [ ]:
import mlflow
import pandas as pd
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall
)
from ragas import evaluate
from datasets import Dataset

def run_evaluation_pipeline(dataset_path: str, run_name: str = "RAG_Eval_Run"):
    df = pd.read_csv(dataset_path)  # Or pd.read_json()

    ragas_dataset = Dataset.from_pandas(df)

    metric_list = [answer_relevancy, context_precision, context_recall, faithfulness]

    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("dataset", dataset_path)
        mlflow.log_param("num_samples", len(df))

        results = evaluate(ragas_dataset, metrics=metric_list)

        for metric in metric_list:
            score = results[metric.name]
            mlflow.log_metric(metric.name, score)

        results_df = pd.DataFrame([results])
        results_df.to_csv("results.csv", index=False)
        mlflow.log_artifact("results.csv")

        print("Evaluation completed and logged to MLflow.")

run_evaluation_pipeline("rag_evaluation_data.csv")